In [35]:
import requests
import pandas as pd
import numpy as np
import json

In [36]:
counter = 0

In [37]:
#Query all data from a year
#Add counter for the number of extractions
data=[]
for i in range(1,30):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2009","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
data
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


['{"get":"players","parameters":{"league":"39","page":"1","season":"2009"},"errors":[],"results":20,"paging":{"current":1,"total":32},"response":[{"player":{"id":468,"name":"V. \\u0106orluka","firstname":"Vedran","lastname":"\\u0106orluka","age":35,"birth":{"date":"1986-02-05","place":"Derventa","country":"Bosnia and Herzegovina"},"nationality":"Croatia","height":"192 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/468.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":null,"name":"Premier League","country":null,"logo":null,"flag":null,"season":"2009-2010"},"games":{"appearences":29,"lineups":29,"minutes":2556,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":0,"out":3,"bench":1},"shots":{"total":null,"on":null},"goals":{"total":1,"conceded":null,"assists":null,"saves":null},"passes":{"total":null,"key":null,

In [38]:
counter

29

In [39]:
#complete year array
for i in range(30,33):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2009","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)
    
data

['{"get":"players","parameters":{"league":"39","page":"1","season":"2009"},"errors":[],"results":20,"paging":{"current":1,"total":32},"response":[{"player":{"id":468,"name":"V. \\u0106orluka","firstname":"Vedran","lastname":"\\u0106orluka","age":35,"birth":{"date":"1986-02-05","place":"Derventa","country":"Bosnia and Herzegovina"},"nationality":"Croatia","height":"192 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/468.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":null,"name":"Premier League","country":null,"logo":null,"flag":null,"season":"2009-2010"},"games":{"appearences":29,"lineups":29,"minutes":2556,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":0,"out":3,"bench":1},"shots":{"total":null,"on":null},"goals":{"total":1,"conceded":null,"assists":null,"saves":null},"passes":{"total":null,"key":null,

In [40]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

32

In [42]:
len(jsonData[31]['response'])

6

In [43]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [44]:
#finalPageItems
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
finalPageItems

6

In [45]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
for i in range(pagesNum):
    
    #To adjust for the last page
    if i == pagesNum-1: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            print(i,j,pkey)
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

0 0 id
0 0 firstname
0 0 lastname
0 0 age
0 0 nationality
0 0 height
0 0 weight
0 0 injured
0 1 id
0 1 firstname
0 1 lastname
0 1 age
0 1 nationality
0 1 height
0 1 weight
0 1 injured
0 2 id
0 2 firstname
0 2 lastname
0 2 age
0 2 nationality
0 2 height
0 2 weight
0 2 injured
0 3 id
0 3 firstname
0 3 lastname
0 3 age
0 3 nationality
0 3 height
0 3 weight
0 3 injured
0 4 id
0 4 firstname
0 4 lastname
0 4 age
0 4 nationality
0 4 height
0 4 weight
0 4 injured
0 5 id
0 5 firstname
0 5 lastname
0 5 age
0 5 nationality
0 5 height
0 5 weight
0 5 injured
0 6 id
0 6 firstname
0 6 lastname
0 6 age
0 6 nationality
0 6 height
0 6 weight
0 6 injured
0 7 id
0 7 firstname
0 7 lastname
0 7 age
0 7 nationality
0 7 height
0 7 weight
0 7 injured
0 8 id
0 8 firstname
0 8 lastname
0 8 age
0 8 nationality
0 8 height
0 8 weight
0 8 injured
0 9 id
0 9 firstname
0 9 lastname
0 9 age
0 9 nationality
0 9 height
0 9 weight
0 9 injured
0 10 id
0 10 firstname
0 10 lastname
0 10 age
0 10 nationality
0 10 height
0 10 

4 11 nationality
4 11 height
4 11 weight
4 11 injured
4 12 id
4 12 firstname
4 12 lastname
4 12 age
4 12 nationality
4 12 height
4 12 weight
4 12 injured
4 13 id
4 13 firstname
4 13 lastname
4 13 age
4 13 nationality
4 13 height
4 13 weight
4 13 injured
4 14 id
4 14 firstname
4 14 lastname
4 14 age
4 14 nationality
4 14 height
4 14 weight
4 14 injured
4 15 id
4 15 firstname
4 15 lastname
4 15 age
4 15 nationality
4 15 height
4 15 weight
4 15 injured
4 16 id
4 16 firstname
4 16 lastname
4 16 age
4 16 nationality
4 16 height
4 16 weight
4 16 injured
4 17 id
4 17 firstname
4 17 lastname
4 17 age
4 17 nationality
4 17 height
4 17 weight
4 17 injured
4 18 id
4 18 firstname
4 18 lastname
4 18 age
4 18 nationality
4 18 height
4 18 weight
4 18 injured
4 19 id
4 19 firstname
4 19 lastname
4 19 age
4 19 nationality
4 19 height
4 19 weight
4 19 injured
5 0 id
5 0 firstname
5 0 lastname
5 0 age
5 0 nationality
5 0 height
5 0 weight
5 0 injured
5 1 id
5 1 firstname
5 1 lastname
5 1 age
5 1 national

9 3 nationality
9 3 height
9 3 weight
9 3 injured
9 4 id
9 4 firstname
9 4 lastname
9 4 age
9 4 nationality
9 4 height
9 4 weight
9 4 injured
9 5 id
9 5 firstname
9 5 lastname
9 5 age
9 5 nationality
9 5 height
9 5 weight
9 5 injured
9 6 id
9 6 firstname
9 6 lastname
9 6 age
9 6 nationality
9 6 height
9 6 weight
9 6 injured
9 7 id
9 7 firstname
9 7 lastname
9 7 age
9 7 nationality
9 7 height
9 7 weight
9 7 injured
9 8 id
9 8 firstname
9 8 lastname
9 8 age
9 8 nationality
9 8 height
9 8 weight
9 8 injured
9 9 id
9 9 firstname
9 9 lastname
9 9 age
9 9 nationality
9 9 height
9 9 weight
9 9 injured
9 10 id
9 10 firstname
9 10 lastname
9 10 age
9 10 nationality
9 10 height
9 10 weight
9 10 injured
9 11 id
9 11 firstname
9 11 lastname
9 11 age
9 11 nationality
9 11 height
9 11 weight
9 11 injured
9 12 id
9 12 firstname
9 12 lastname
9 12 age
9 12 nationality
9 12 height
9 12 weight
9 12 injured
9 13 id
9 13 firstname
9 13 lastname
9 13 age
9 13 nationality
9 13 height
9 13 weight
9 13 injure

13 11 lastname
13 11 age
13 11 nationality
13 11 height
13 11 weight
13 11 injured
13 12 id
13 12 firstname
13 12 lastname
13 12 age
13 12 nationality
13 12 height
13 12 weight
13 12 injured
13 13 id
13 13 firstname
13 13 lastname
13 13 age
13 13 nationality
13 13 height
13 13 weight
13 13 injured
13 14 id
13 14 firstname
13 14 lastname
13 14 age
13 14 nationality
13 14 height
13 14 weight
13 14 injured
13 15 id
13 15 firstname
13 15 lastname
13 15 age
13 15 nationality
13 15 height
13 15 weight
13 15 injured
13 16 id
13 16 firstname
13 16 lastname
13 16 age
13 16 nationality
13 16 height
13 16 weight
13 16 injured
13 17 id
13 17 firstname
13 17 lastname
13 17 age
13 17 nationality
13 17 height
13 17 weight
13 17 injured
13 18 id
13 18 firstname
13 18 lastname
13 18 age
13 18 nationality
13 18 height
13 18 weight
13 18 injured
13 19 id
13 19 firstname
13 19 lastname
13 19 age
13 19 nationality
13 19 height
13 19 weight
13 19 injured
14 0 id
14 0 firstname
14 0 lastname
14 0 age
14 0 na

18 1 height
18 1 weight
18 1 injured
18 2 id
18 2 firstname
18 2 lastname
18 2 age
18 2 nationality
18 2 height
18 2 weight
18 2 injured
18 3 id
18 3 firstname
18 3 lastname
18 3 age
18 3 nationality
18 3 height
18 3 weight
18 3 injured
18 4 id
18 4 firstname
18 4 lastname
18 4 age
18 4 nationality
18 4 height
18 4 weight
18 4 injured
18 5 id
18 5 firstname
18 5 lastname
18 5 age
18 5 nationality
18 5 height
18 5 weight
18 5 injured
18 6 id
18 6 firstname
18 6 lastname
18 6 age
18 6 nationality
18 6 height
18 6 weight
18 6 injured
18 7 id
18 7 firstname
18 7 lastname
18 7 age
18 7 nationality
18 7 height
18 7 weight
18 7 injured
18 8 id
18 8 firstname
18 8 lastname
18 8 age
18 8 nationality
18 8 height
18 8 weight
18 8 injured
18 9 id
18 9 firstname
18 9 lastname
18 9 age
18 9 nationality
18 9 height
18 9 weight
18 9 injured
18 10 id
18 10 firstname
18 10 lastname
18 10 age
18 10 nationality
18 10 height
18 10 weight
18 10 injured
18 11 id
18 11 firstname
18 11 lastname
18 11 age
18 11

22 12 age
22 12 nationality
22 12 height
22 12 weight
22 12 injured
22 13 id
22 13 firstname
22 13 lastname
22 13 age
22 13 nationality
22 13 height
22 13 weight
22 13 injured
22 14 id
22 14 firstname
22 14 lastname
22 14 age
22 14 nationality
22 14 height
22 14 weight
22 14 injured
22 15 id
22 15 firstname
22 15 lastname
22 15 age
22 15 nationality
22 15 height
22 15 weight
22 15 injured
22 16 id
22 16 firstname
22 16 lastname
22 16 age
22 16 nationality
22 16 height
22 16 weight
22 16 injured
22 17 id
22 17 firstname
22 17 lastname
22 17 age
22 17 nationality
22 17 height
22 17 weight
22 17 injured
22 18 id
22 18 firstname
22 18 lastname
22 18 age
22 18 nationality
22 18 height
22 18 weight
22 18 injured
22 19 id
22 19 firstname
22 19 lastname
22 19 age
22 19 nationality
22 19 height
22 19 weight
22 19 injured
23 0 id
23 0 firstname
23 0 lastname
23 0 age
23 0 nationality
23 0 height
23 0 weight
23 0 injured
23 1 id
23 1 firstname
23 1 lastname
23 1 age
23 1 nationality
23 1 height
2

26 12 age
26 12 nationality
26 12 height
26 12 weight
26 12 injured
26 13 id
26 13 firstname
26 13 lastname
26 13 age
26 13 nationality
26 13 height
26 13 weight
26 13 injured
26 14 id
26 14 firstname
26 14 lastname
26 14 age
26 14 nationality
26 14 height
26 14 weight
26 14 injured
26 15 id
26 15 firstname
26 15 lastname
26 15 age
26 15 nationality
26 15 height
26 15 weight
26 15 injured
26 16 id
26 16 firstname
26 16 lastname
26 16 age
26 16 nationality
26 16 height
26 16 weight
26 16 injured
26 17 id
26 17 firstname
26 17 lastname
26 17 age
26 17 nationality
26 17 height
26 17 weight
26 17 injured
26 18 id
26 18 firstname
26 18 lastname
26 18 age
26 18 nationality
26 18 height
26 18 weight
26 18 injured
26 19 id
26 19 firstname
26 19 lastname
26 19 age
26 19 nationality
26 19 height
26 19 weight
26 19 injured
27 0 id
27 0 firstname
27 0 lastname
27 0 age
27 0 nationality
27 0 height
27 0 weight
27 0 injured
27 1 id
27 1 firstname
27 1 lastname
27 1 age
27 1 nationality
27 1 height
2

In [46]:
len(finalArr)

626

In [51]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[468, 'Vedran', 'Ćorluka', ..., None, None, None],
       [5982, 'Carlos Alberto', 'Tevez', ..., None, None, None],
       [15441, 'Abdisalam', 'Ibrahim', ..., None, None, None],
       ...,
       [70725, 'Paulo César', 'Da Silva Barrios', ..., None, None, None],
       [70775, 'Roque Luis', 'Santa Cruz Cantero', ..., None, None, None],
       [87295, 'Steed', 'Malbranque', ..., None, None, None]],
      dtype=object)

In [54]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df[df['lastname']=='Çiftçi']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
539,61824,Nadir,Çiftçi,30,Turkey,187 cm,77 kg,False,1355,Portsmouth,...,None,None,0,0,0,None,None,None,None,None


In [50]:
gb = df.groupby(df['lastname']).count()#=='Kane'].goalAssists
df[df['lastname']=='Çiftçi']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved,season
599,51146,Vito,Mannone,34,Italy,191 cm,73 kg,False,42,Arsenal,...,None,0,0,0,None,None,None,None,None,2010/2011


In [32]:
df['season'] = '2009/2010'

In [33]:
#Export the df to a csv
df.to_csv('data2009&2010.csv')

In [34]:
counter

70